<a href="https://colab.research.google.com/github/nithishhhhhhh/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Dataset From Kaggle


In [22]:
! pip install kaggle

ERROR: Operation cancelled by user


In [23]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [24]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [25]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print('The dataset extracted')


The dataset extracted


Importing required libraries and dependencies


In [26]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords  #Natural Language Toolkit
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Processing


In [28]:
# csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [29]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


We need to add the column names to the DataFrame because the first line of the dataset contains data but is mistakenly being interpreted as column headers.

In [30]:
#Coloumn names assigning
column_names = ['target','id','data','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding = 'ISO-8859-1')

In [31]:
twitter_data.head()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Finding Missing Values

In [32]:
twitter_data.isnull().sum()

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

Check the distribution of the target column.

In [33]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [34]:
# target '4' is converted to '1'
twitter_data.replace({'target':{4:1}},inplace = True )

In [35]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 ---> Negative tweet

1 ---> Positive tweet

Stemming


In [36]:
port_stem =PorterStemmer()

In [37]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [38]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [39]:
#stemmed content
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [40]:
#Seperating data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [41]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data(80) and test data(20)

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2, stratify=Y, random_state = 2)

In [43]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


Converting Textual data into Numerical with the help of TFIDF VECTORIZER

In [44]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Training the Machine Learning Model - Logistic Regression

In [45]:
model = LogisticRegression(max_iter=1000)

In [46]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation by Accuracy score

In [47]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [48]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.81018984375


In [49]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [50]:
print('Accuracy score on the test data :', test_data_accuracy)

Accuracy score on the test data : 0.7780375


Model Accuracy = 77.8%

Saving the Trained Model for further usage.

In [51]:
import pickle

In [52]:
filename = 'TSA_trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Example

In [53]:
load_model = pickle.load(open('/content/TSA_trained_model.sav','rb'))

In [57]:
X_1 = X_test[2000]
print(Y_test[2000])

prediction = load_model.predict(X_1)
print(prediction)

if (prediction[0] == 0):
  print('Negative tweet')
else:
  print('Positive tweet')

0
[0]
Negative tweet
